In [3]:
from pybaseball import statcast_batter_expected_stats
from pybaseball import pitching_stats
from pybaseball import statcast_batter
from pybaseball import playerid_lookup
import pandas as pd
from scipy.stats import zscore
from pybaseball import cache

## Batters by wBOA ZScore

In [2]:
from pybaseball import batting_stats
# retrieve data on only players who have 50+ plate appearances this year
full_batting_2019 = batting_stats(2019, qual=150)
full_batting_2021 = batting_stats(2021, qual=150)
full_batting_2022 = batting_stats(2022, qual=150)
full_batting_2023 = batting_stats(2023, qual=150)

# Concatenate the DataFrames vertically
combined_batting_data = pd.concat([full_batting_2019,full_batting_2021, full_batting_2022, full_batting_2023], ignore_index=True)
full_batting_2022.head()

,IDfg


In [16]:
print("Shape of the DataFrame:", combined_batting_data.shape)
print("Column names:", combined_batting_data.columns.tolist())

Shape of the DataFrame: (0, 1)
Column names: ['IDfg']


In [14]:
# Filter the DataFrame for the relevant years (2019, 2021, 2022)
relevant_years = [2019, 2021, 2022]
filtered_data_batting = combined_batting_data[combined_batting_data['Season'].isin(relevant_years)]

# Group the data by player and calculate the average wOBA for relevant years
grouped_data = filtered_data_batting.groupby(['IDfg', 'Name']).mean().reset_index().rename(columns={'wOBA': 'avg_wOBA'})

# Merge the average wOBA with the 2023 wOBA for each player
woba_2023 = full_batting_2023[['IDfg', 'Name', 'wOBA']].rename(columns={'wOBA': 'wOBA_2023'})


# Merge player names with the result DataFrame
# Merge player names and calculated results
players_off_output = pd.merge(
    grouped_data,
    woba_2023,
    on=['IDfg', 'Name']
)

KeyError: 'Season'

In [ ]:
# Drop Season Column
players_off_output = players_off_output.drop(columns='Season')
pd.set_option('display.float_format', '{:.2f}'.format)

# Reorder columns to have 'avg_wOBA' and 'wOBA_2023' next to each other at the end
column_order = [
    col for col in players_off_output.columns if col not in ['avg_wOBA', 'wOBA_2023']
] + ['avg_wOBA', 'wOBA_2023']

players_off_output = players_off_output[column_order]

In [ ]:
avg_wOBA = players_off_output['avg_wOBA']
wOBA_2023 = players_off_output['wOBA_2023']

z_scores_avg_woba = zscore(avg_wOBA)
z_score_woba_2023 = zscore(wOBA_2023)

In [ ]:
# # Calculate z-scores for avg_wOBA and wOBA_2023
# players_off_output['zscore_avg_wOBA'] = z_scores_avg_woba 
# players_off_output['zscore_wOBA_2023'] = z_score_woba_2023

# Calculate the difference between z-scores
# players_off_output['zscore_difference'] = players_off_output['zscore_wOBA_2023'] - players_off_output['zscore_avg_wOBA']

players_off_output['zscore_difference'] = z_score_woba_2023 - z_scores_avg_woba 
# Display the result
players_off_output

In [ ]:
 #Write the DataFrame to a CSV file
players_off_output.to_csv('Resources/off_output_for_learning.csv', index=False)

## Pitchers by zERA ZScore

In [7]:
from pybaseball import statcast_pitcher_expected_stats

# Get data for all pitchers with a minimum of 150 plate appearances against in  2019, 2021, 2022, 2023
pitcher_data_2019 = statcast_pitcher_expected_stats(2019, 100)
pitcher_data_2021 = statcast_pitcher_expected_stats(2021, 150)
pitcher_data_2022 = statcast_pitcher_expected_stats(2022, 150)
pitcher_data_2023 = statcast_pitcher_expected_stats(2023, 150)

# Concatenate the DataFrames
pitcher_combined_data = pd.concat([pitcher_data_2019, pitcher_data_2021, pitcher_data_2022, pitcher_data_2023], ignore_index=True)

# # Concatenate the last_name and first_name columns to create a new 'Name' column
# pitcher_combined_data['Name'] = pitcher_combined_data['first_name'] + ' ' + pitcher_combined_data['last_name']

# # Drop the individual 'last_name' and 'first_name' columns
# pitcher_combined_data.drop(columns=['last_name', 'first_name'], inplace=True)

# Display the modified DataFrame
pitcher_combined_data_ = pitcher_combined_data.drop(['last_name', 'first_name'], axis=1)
pitcher_combined_data.to_csv('Resources/pitcher_combined_data.csv', index=False)

In [ ]:
# pitcher_data_2023['Name'] = pitcher_data_2023['first_name'] + ' ' + pitcher_data_2023['last_name']

# # Drop the individual 'last_name' and 'first_name' columns
# pitcher_data_2023.drop(columns=['last_name', 'first_name'], inplace=True)

# # Display the modified DataFrame
# pitcher_data_2023

In [11]:
relevant_years = [2019, 2021, 2022]
filtered_data = pitcher_combined_data[pitcher_combined_data['year'].isin(relevant_years)]

pitching_data_2023 = pitcher_combined_data[pitcher_combined_data['year'] == 2023]
# Merge the average wOBA with the 2023 wOBA for each player
grouped_data = filtered_data.groupby(['player_id']).mean().reset_index().rename(columns={'xera': 'avg_zERA'})

grouped_data

/var/folders/pn/c8dt84bx2r9fbfxb1qyp16k80000gn/T/ipykernel_1895/1639892875.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grouped_data = filtered_data.groupby(['player_id']).mean().reset_index().rename(columns={'xera': 'avg_zERA'})


,player_id,year,pa,bip,ba,est_ba,est_ba_minus_ba_diff,slg,est_slg,est_slg_minus_slg_diff,woba,est_woba,est_woba_minus_woba_diff,era,avg_zERA,era_minus_xera_diff
0,282332,2019.000000,468.000000,318.000000,0.265000,0.242,0.023000,0.512000,0.421000,0.091,0.351000,0.315000,0.036000,4.95,4.380000,0.567000
1,425794,2020.666667,792.000000,571.333333,0.251333,0.262,-0.010667,0.391333,0.412333,-0.021,0.303000,0.319667,-0.016667,3.65,4.396667,-0.744333
2,425844,2020.666667,697.333333,536.666667,0.255333,0.263,-0.007667,0.405000,0.420000,-0.015,0.297667,0.312667,-0.015000,3.59,4.196667,-0.606667
3,429719,2019.000000,336.000000,249.000000,0.351000,0.309,0.042000,0.686000,0.592000,0.094,0.449000,0.408000,0.041000,9.58,8.040000,1.536000
4,429722,2021.000000,277.000000,201.000000,0.259000,0.278,-0.019000,0.446000,0.533000,-0.087,0.328000,0.369000,-0.041000,4.68,6.030000,-1.346000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,680739,2022.000000,287.000000,217.000000,0.264000,0.268,-0.004000,0.460000,0.457000,0.003,0.337000,0.349000,-0.012000,4.70,5.210000,-0.509000
541,682051,2022.000000,263.000000,190.000000,0.231000,0.270,-0.039000,0.380000,0.471000,-0.091,0.316000,0.364000,-0.048000,4.03,5.750000,-1.722000
542,682171,2022.000000,272.000000,176.000000,0.192000,0.200,-0.008000,0.320000,0.316000,0.004,0.251000,0.254000,-0.003000,2.99,2.640000,0.346000
543,689225,2022.000000,339.000000,259.000000,0.236000,0.286,-0.050000,0.411000,0.490000,-0.079,0.306000,0.360000,-0.054000,4.19,5.600000,-1.412000


In [12]:
zera_2023 = pitcher_data_2023[['player_id', 'xera']].rename(columns={'xera': 'zERA_2023'})
zera_2023

,player_id,zERA_2023
0,645261,4.08
1,657277,3.54
2,668678,4.06
3,656605,4.03
4,605400,3.86
...,...,...
199,669467,3.86
200,671345,3.59
201,660813,3.24
202,641793,4.68


In [9]:
# Merge player names with the result DataFrame
# Merge player names and calculated results
pitcher_output = pd.merge(
    grouped_data,
    zera_2023,
    on=['player_id']
)

# Display the result
pitcher_output

,player_id,year,pa,bip,ba,est_ba,est_ba_minus_ba_diff,slg,est_slg,est_slg_minus_slg_diff,woba,est_woba,est_woba_minus_woba_diff,era,avg_zERA,era_minus_xera_diff,zERA_2023
0,425794,2020.666667,792.000000,571.333333,0.251333,0.2620,-0.010667,0.391333,0.412333,-0.0210,0.303000,0.319667,-0.016667,3.650,4.396667,-0.744333,7.69
1,425844,2020.666667,697.333333,536.666667,0.255333,0.2630,-0.007667,0.405000,0.420000,-0.0150,0.297667,0.312667,-0.015000,3.590,4.196667,-0.606667,5.39
2,434378,2020.500000,756.500000,472.000000,0.179000,0.1965,-0.017500,0.315500,0.338500,-0.0230,0.232500,0.251500,-0.019000,2.165,2.660000,-0.494000,3.54
3,446372,2021.500000,515.000000,370.000000,0.260500,0.2505,0.010000,0.400000,0.394000,0.0060,0.313000,0.309500,0.003500,4.080,4.015000,0.065000,6.17
4,448179,2021.500000,593.500000,408.000000,0.246500,0.2400,0.006500,0.417000,0.412500,0.0045,0.319000,0.318000,0.001000,4.065,4.265000,-0.201000,5.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,677960,2021.000000,401.000000,296.000000,0.279000,0.2770,0.002000,0.497000,0.474000,0.0230,0.356000,0.349000,0.007000,5.320,5.300000,0.024000,4.91
149,678394,2022.000000,268.000000,184.000000,0.315000,0.2470,0.068000,0.416000,0.347000,0.0690,0.355000,0.303000,0.052000,4.710,3.800000,0.909000,4.20
150,680686,2021.500000,478.000000,308.000000,0.238500,0.2190,0.019500,0.497500,0.436500,0.0610,0.349000,0.322000,0.027000,5.255,4.380000,0.870500,4.72
151,680694,2022.000000,509.000000,344.000000,0.265000,0.2530,0.012000,0.419000,0.404000,0.0150,0.334000,0.327000,0.007000,4.890,4.490000,0.405000,4.06


In [13]:
pitcher_output = pitcher_output.drop(columns='year')
pd.set_option('display.float_format', '{:.2f}'.format)

In [14]:
avg_zera = pitcher_output['avg_zERA']
zera_2023 = pitcher_output['zERA_2023']
# 
z_scores_avg_zera = zscore(avg_zera)
z_score_zera_2023 = zscore(zera_2023)

In [17]:
pitcher_output['zscore_difference'] = z_score_zera_2023 - z_scores_avg_zera
# Display the result

column_order = [
    'player_id', 'pa', 'bip', 'ba', 'est_ba', 'est_ba_minus_ba_diff',
    'slg', 'est_slg', 'est_slg_minus_slg_diff', 'woba', 'est_woba',
    'est_woba_minus_woba_diff', 'era', 'era_minus_xera_diff', 'avg_zERA',
    'zERA_2023', 'zscore_difference'
]

# Reorder the columns
pitcher_output = pitcher_output[column_order]

# Display the result
pitcher_output

,player_id,pa,bip,ba,est_ba,est_ba_minus_ba_diff,slg,est_slg,est_slg_minus_slg_diff,woba,est_woba,est_woba_minus_woba_diff,era,era_minus_xera_diff,avg_zERA,zERA_2023,zscore_difference
0,425794,792.00,571.33,0.25,0.26,-0.01,0.39,0.41,-0.02,0.30,0.32,-0.02,3.65,-0.74,4.40,7.69,3.17
1,425844,697.33,536.67,0.26,0.26,-0.01,0.40,0.42,-0.01,0.30,0.31,-0.01,3.59,-0.61,4.20,5.39,0.87
2,434378,756.50,472.00,0.18,0.20,-0.02,0.32,0.34,-0.02,0.23,0.25,-0.02,2.17,-0.49,2.66,3.54,1.02
3,446372,515.00,370.00,0.26,0.25,0.01,0.40,0.39,0.01,0.31,0.31,0.00,4.08,0.07,4.02,6.17,2.01
4,448179,593.50,408.00,0.25,0.24,0.01,0.42,0.41,0.00,0.32,0.32,0.00,4.06,-0.20,4.27,5.49,0.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,677960,401.00,296.00,0.28,0.28,0.00,0.50,0.47,0.02,0.36,0.35,0.01,5.32,0.02,5.30,4.91,-1.28
149,678394,268.00,184.00,0.32,0.25,0.07,0.42,0.35,0.07,0.35,0.30,0.05,4.71,0.91,3.80,4.20,0.11
150,680686,478.00,308.00,0.24,0.22,0.02,0.50,0.44,0.06,0.35,0.32,0.03,5.26,0.87,4.38,4.72,-0.15
151,680694,509.00,344.00,0.27,0.25,0.01,0.42,0.40,0.01,0.33,0.33,0.01,4.89,0.41,4.49,4.06,-1.06


In [ ]:
 #Write the DataFrame to a CSV file
pitcher_output.to_csv('Resources/pitcher_output_for_leaning.csv', index=False)